In [54]:
# mlflow server --backend-store-uri sqlite:///path/to/mlflow.db --default-artifact-root ./mlruns --host 0.0.0.0 --port 5000

In [46]:
# mlflow models serve -m "4/064c1ab8713c40149f92bd6849cee7fc/artifacts/logistic_regression/" --no-conda --host 0.0.0.0 --port 5001

In [47]:
# kill -9 $(lsof -ti :5000)

In [14]:
import numpy as np
import requests
import json
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

In [15]:
data = fetch_covtype()
X, y = data.data, data.target

In [16]:
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y

In [17]:
df_sampled = df.sample(n=1000, random_state=42)

In [18]:
X_sampled = df_sampled.drop(columns=['target'])
y_sampled = df_sampled['target']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42, stratify=y_sampled)

In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, solver='saga'),
    "Decision Tree": DecisionTreeClassifier(max_depth=10),
    "CatBoost": CatBoostClassifier(verbose=0)
}

In [22]:
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name}: Accuracy = {acc:.4f}")

Logistic Regression: Accuracy = 0.7350
Decision Tree: Accuracy = 0.6400
CatBoost: Accuracy = 0.7800


In [23]:
# mlflow.set_tracking_uri("http://127.0.0.1:5000")
# mlflow.create_experiment("new_experiment")
# mlflow.set_experiment("new_experiment")

In [27]:
# Start MLflow experiment
mlflow.set_experiment("experiment_2")

<Experiment: artifact_location='/home/chen/Desktop/mlruns/3', creation_time=1738836534702, experiment_id='3', last_update_time=1738836534702, lifecycle_stage='active', name='experiment_2', tags={}>

In [24]:
input_example = X_train[0]

In [37]:
input_example = input_example.reshape(1, -1)

In [30]:

results = {}
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        # Log model parameters
        if hasattr(model, "get_params"):
            params = model.get_params()
            mlflow.log_params(params)

        # Log additional metadata
        mlflow.log_param("model_name", name)
        mlflow.log_param("dataset_size", len(X_sampled))
        mlflow.log_param("test_size_ratio", 0.2)
        mlflow.log_param("feature_scaling", "StandardScaler")

        # Log metrics
        mlflow.log_metric("accuracy", acc)

        # Log model itself
        mlflow.sklearn.log_model(model, name, input_example=input_example)

        results[name] = acc
        print(f"{name}: Accuracy = {acc:.4f}")

print("\n✅ Experiment tracking complete. Run `mlflow ui` to view results.")


Logistic Regression: Accuracy = 0.7350
🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/3/runs/b49f402432bc4e52880f8d6f7e5aff34
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
Decision Tree: Accuracy = 0.6800
🏃 View run Decision Tree at: http://127.0.0.1:5000/#/experiments/3/runs/c87947d63ba54c8a94998208b690b972
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
CatBoost: Accuracy = 0.7800
🏃 View run CatBoost at: http://127.0.0.1:5000/#/experiments/3/runs/6a466b77ed2644b6b66ea36366e9ccff
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3

✅ Experiment tracking complete. Run `mlflow ui` to view results.


In [31]:
print(mlflow.get_tracking_uri())

http://127.0.0.1:5000


# mlflow Registry

In [34]:
mlflow.set_experiment("experiment_3")

2025/02/06 12:09:29 INFO mlflow.tracking.fluent: Experiment with name 'experiment_3' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/chen/Desktop/mlruns/4', creation_time=1738836569932, experiment_id='4', last_update_time=1738836569932, lifecycle_stage='active', name='experiment_3', tags={}>

In [35]:
mdl = LogisticRegression()
mdl.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [36]:
acc = accuracy_score(y_test, y_pred)
print(acc)

0.78


In [37]:
model = mdl

with mlflow.start_run():
    # Log model parameters (optional)
    mlflow.log_param("model_type", "logistic_regression")

    # Log the trained model
    model_uri = "logistic_regression"
    mlflow.sklearn.log_model(model, model_uri, input_example=input_example)

    # Register the model in the MLflow Model Registry
    result = mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/{model_uri}", "logistic_regression")

print(f"Model registered as: {result.name}, version: {result.version}")


Successfully registered model 'logistic_regression'.
2025/02/06 12:09:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic_regression, version 1


🏃 View run crawling-ray-512 at: http://127.0.0.1:5000/#/experiments/4/runs/064c1ab8713c40149f92bd6849cee7fc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
Model registered as: logistic_regression, version: 1


Created version '1' of model 'logistic_regression'.


In [41]:
mlflow.get_tracking_uri()

'http://127.0.0.1:5000'

In [42]:

client = MlflowClient()
model_versions = client.get_latest_versions("logistic_regression")
for v in model_versions:
    print(f"Version {v.version} - {v.current_stage}")


Version 1 - None


/tmp/ipykernel_58213/2109146132.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  model_versions = client.get_latest_versions("logistic_regression")


In [43]:
input_example

array([[ 0.31905034,  0.92898242,  0.78444028, -0.38050697, -0.63090735,
        -1.06414993, -1.62927445,  1.07074037,  1.89420581, -0.77028791,
        -0.93228691, -0.22021079,  1.17815946, -0.27435163, -0.07930516,
        -0.1334605 , -0.09395338, -0.15597367, -0.05006262, -0.09395338,
        -0.03537746,  0.        ,  0.        , -0.27170849, -0.10666772,
        -0.24413654,  7.        , -0.03537746,  0.        , -0.05006262,
        -0.07088812, -0.05006262, -0.10666772, -0.1334605 ,  0.        ,
        -0.27697656, -0.33333333, -0.21068086,  0.        , -0.03537746,
        -0.08692914,  0.        , -0.5       , -0.23241869, -0.21707238,
        -0.31207962, -0.27170849, -0.05006262, -0.03537746,  0.        ,
        -0.03537746, -0.1382327 , -0.1382327 , -0.12852402]])

In [44]:
mdl.predict(input_example)

array([2], dtype=int32)

In [52]:
url = "http://localhost:5001/invocations"

data = json.dumps({"instances": input_example.tolist()})

headers = {"Content-Type": "application/json"}

In [53]:
response = requests.post(url, headers=headers, data=data)

print(response.json())

{'predictions': [2]}
